In [1]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from IPython.display import Javascript, display
from ipywidgets import Text, HBox, VBox, Button

authors = widgets.Text(value='Author name',placeholder='Type something', description="author")
display(authors)


button = widgets.Button(description="Update names")
display(button)

# Add author names to the list and refreshing next cells
author_names = []
def on_button_clicked(b):
    author_names.append(authors.value)
def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
button.on_click(on_button_clicked)
button.on_click(run_all)

Text(value='Author name', description='author', placeholder='Type something')

Button(description='Update names', style=ButtonStyle())

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
#create the name_dict with values Task_value and Task
name_dict={"Task_value" : pd.Series([],dtype="float32"),
          "Task" : pd.Series([],dtype="str")}

# create the input_contribution for all authors
input_contribution = {"Task" : ["Task"], "Task_value" : ["Task_value"]}
for i in range(0,len(author_names)):
    input_contribution.update({author_names[i] : [author_names[i]]})

# add value to input_contribution dict
def on_button2_clicked(b):
    for i in list(input_contribution):
        input_contribution[i].append(0)
       
button2 = widgets.Button(description="Add row")
button2.on_click(on_button2_clicked)
button2.on_click(run_all)
display(button2)

Button(description='Add row', style=ButtonStyle())

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:

#define vertical boxes
all_box=[]
for i in list(input_contribution):
    all_box.append(input_contribution[i])

#divide all_box into a lists of Text objects
v_box=[]
input_contribution_list=dict()
for i in range(0, len(input_contribution)):
    all_box_str = [str(x) for x in all_box[i]]   
    input_contribution_list.update({list(input_contribution)[i] : [Text(value=w, placeholder='Type something') for w in all_box_str]})
    v_box.append(VBox(input_contribution_list[list(input_contribution)[i]]))    

# show array of boxes
HBox(v_box)

HBox(children=(VBox(children=(Text(value='Task', placeholder='Type something'), Text(value='0', placeholder='Type something'), Text(value='0', placeholder='Type something'))), VBox(children=(Text(value='Task_value', placeholder='Type something'), Text(value='0', placeholder='Type something'), Text(value='0', placeholder='Type something'))), VBox(children=(Text(value='au1', placeholder='Type something'), Text(value='0', placeholder='Type something'), Text(value='0', placeholder='Type something'))), VBox(children=(Text(value='au2', placeholder='Type something'), Text(value='0', placeholder='Type something'), Text(value='0', placeholder='Type something')))))

In [40]:
# extract values from input_contribution_list dict
output_lists=[]
for i in list(input_contribution):
    output_lists.append([x.value for x in input_contribution_list[i]])
# transpose output_lists
output_lists = list(map(list, zip(*output_lists)))
# create output_contribution df
output_contribution = pd.DataFrame(output_lists[1:],columns=output_lists[0])

# convert numbers back to float
output_contribution.iloc[:,1:len(list(output_contribution))] = output_contribution.iloc[:,1:len(list(output_contribution))].applymap(float)

# normalize Task_values to 100%
output_contribution["Task_value"]=[x/output_contribution["Task_value"].sum() for x in output_contribution["Task_value"]]

# normalize author contributions to 100%
for n in output_contribution.index:
    output_contribution.loc[n,author_names] = [x/output_contribution.loc[n,author_names].sum() for x in output_contribution.loc[n,author_names]]

# multiply author contributions with Task_values
for n in output_contribution.index:
    output_contribution.loc[n,author_names] = [output_contribution.loc[n,"Task_value"] * x for x in output_contribution.loc[n,author_names]]
    
# create row for scores and fill them with corresponding values
results={"Task" : pd.Series(["Score"]), "Task_value" : output_contribution["Task_value"].sum()}
for i in range(0,len(author_names)):
    results[author_names[i]]=pd.Series(output_contribution[author_names[i]].sum())
dresults = pd.DataFrame(results,index=list(range(1)))

# add scores to contribution matrix
output_contribution = output_contribution.append(dresults)

# multiply by 100 to get %
output_contribution[author_names]=round(output_contribution[author_names]*100)
output_contribution["Task_value"]=round(output_contribution["Task_value"]*100)
display(output_contribution)

,Task,Task_value,au1,au2
0,experimten,40.0,17.1429,22.8571
1,writing,60.0,24,36
0,Score,100.0,41.1429,58.8571
